In [3]:
import gymnasium as gym 
from gymnasium import spaces
import numpy as np
import torch as th
import os
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline
from stable_baselines3 import SAC
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback
import pickle
from concurrent.futures import ProcessPoolExecutor

In [4]:
class MAB_Smart(gym.Env):
    """
    聪明的 v.s. 单机的
    用于训练聪明的策略,具有bouns信息,应对返回6*k+1维状态的多臂老虎机环境
    聪明的指：知道赢者通吃规则
    """
    def __init__(self, probs, T,oppo_model):
        """
        初始化多臂老虎机环境
        probs: 每个臂的成功概率
        T: 总时间步数
        """
        super(MAB_Smart, self).__init__()
        self.k = len(probs)  # 臂的数量
        self.probs = probs  # 每个臂的成功概率
        self.T = T  # 总时间步数
        self.oppo_model = oppo_model   # 对手模型

        # 定义动作空间和观察空间
        self.action_space = spaces.Box(low=1, high=100, shape=(1,), dtype=np.float32)  # 动作空间为alpha\in[1,100]代指UCB算法中的探索因子
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(6 * self.k + 1,))  # 观察空间为(6*k+1)维向量，每个维度包含三个元素：拉臂的次数、获得的奖励、奖励的方差，对手拉臂的次数、对手获得的奖励、对手的方差以及最后一个元素是时间t
        self.reset()

    def reset(self, seed=None, options=None):
        """
        重置环境
        返回: 初始状态
        """
        super().reset(seed=seed)
        self.steps = 0
        self.n = np.zeros(self.k)  # 自己每个臂的拉动次数
        self.average_rewards = np.zeros(self.k)  # 自己拉的每个臂奖励的均值
        self.rewards = [[] for _ in range(self.k)]  # 自己每个臂的奖励列表
        self.var = np.zeros(self.k) # 自己每个臂奖励的方差

        self.oppo_n = np.zeros(self.k)  # 对手每个臂的拉动次数
        self.oppo_average_rewards = np.zeros(self.k)    # 对手每个臂奖励的均值
        self.oppo_rewards = [[] for _ in range(self.k)]    # 对手每个臂的奖励列表
        self.oppo_var = np.zeros(self.k)    # 对手每个臂奖励的方差
        self.oppo_state = np.zeros(6 * self.k + 1) # 对手的状态,包含两人全部的信息

        observation = np.zeros(6 * self.k + 1)  # 初始状态

        self.alpha = 1.0  # self初始alpha值
        self.oppo_alpha = 1.0  # 对手初始alpha值
        info = {}
        return observation, info
    
    def step(self, action):
        """
        执行给定的动作，并返回结果。
        
        参数:
        action: 智能体选择的alpha值
        
        返回:
        - next_state: 执行动作后的新状态
        - reward: 执行动作后获得的奖励
        - terminated: 布尔值，表示是否达到终止状态
        - truncated: 布尔值，表示是否由于时间限制或其他原因而被截断
        - info: 额外的诊断信息（可选）
        """
        self.steps += 1     # 更新步数

        # 计算两人开局时面对的汇总信息
        total_n = self.n + self.oppo_n  # 总的拉动次数
        total_mean = (self.n * self.average_rewards + self.oppo_n * self.oppo_average_rewards) / (total_n + 1e-10)  # 总的奖励均值

        # 自己行动
        self.alpha = action[0]  # 更新alpha值
        self_ucb_values = total_mean + np.sqrt(self.alpha * np.log(2 * self.steps + 1) / (2 * (total_n + 1e-10)))  # 根据汇总信息,计算自己UCB值
        self_chosen_arm = np.argmax(self_ucb_values)  # 选择具有最高UCB值的臂

        # 自己该次行动的影响: 更新前3*k维的状态
        reward = np.random.binomial(1, self.probs[self_chosen_arm])  # 根据奖励概率生成奖励
        self.n[self_chosen_arm] += 1
        self.rewards[self_chosen_arm].append(reward)  # 更新奖励列表
        self.average_rewards[self_chosen_arm] = np.mean(self.rewards[self_chosen_arm])  # 更新奖励均值
        self.var[self_chosen_arm] = np.var(self.rewards[self_chosen_arm])#更新奖励方差

        # 对手行动:根据上局结束时的状态进行predict，采用上一个时刻面对的全部的信息来选择臂
        oppo_state_tensor = th.tensor(self.oppo_state, dtype=th.float32).unsqueeze(0) # 将上局结束时的状态转换为张量
        oppo_action, _ = self.oppo_model.predict(oppo_state_tensor, deterministic=True)  # 根据上局结束的状态，对手给出预测的alpha值
        self.oppo_alpha = oppo_action[0] # 对手的alpha值
        oppo_ucb_values = total_mean + np.sqrt(self.oppo_alpha * np.log(2*self.steps + 1) / (2 * (total_n + 1e-10))) # 根据汇总信息,计算对手UCB值
        oppo_chosen_arm = np.argmax(oppo_ucb_values) # 对手选择具有最高UCB值的臂

        # 对手该次行动的影响: 只影响对手
        oppo_reward = np.random.binomial(1, self.probs[oppo_chosen_arm]) # 根据奖励概率生成奖励
        self.oppo_n[oppo_chosen_arm] += 1 # 更新对手的拉动次数
        self.oppo_rewards[oppo_chosen_arm].append(oppo_reward) # 更新对手的奖励列表
        self.oppo_average_rewards[oppo_chosen_arm] = np.mean(self.oppo_rewards[oppo_chosen_arm])    # 更新对手的奖励均值
        self.oppo_var[oppo_chosen_arm] = np.var(self.oppo_rewards[oppo_chosen_arm])     # 更新对手的奖励方差


        #   更新对手的状态，前3*k维为对手的信息，后3*k维为self的信息
        self.oppo_state = np.concatenate([self.oppo_n, self.oppo_average_rewards, self.oppo_var, self.n, self.average_rewards, self.var, [self.steps]]) # 更新对手的状态,包含两人全部的信息


        
        terminated = self.steps >= self.T  # 判断是否达到最大步数
        truncated = self.steps >= self.T  # 在这个示例中，截断条件与终止条件相同
        next_state = np.concatenate([self.n, self.average_rewards, self.var, self.oppo_n, self.oppo_average_rewards, self.oppo_var, [self.steps]])  # 更新自身状态,包含两人全部的信息
        info = {} # 可选的额外信息
        if terminated:
            self_total_rewards = sum(sum(rewards) for rewards in self.rewards)
            oppo_total_rewards = sum(sum(rewards) for rewards in self.oppo_rewards)
            if self_total_rewards > oppo_total_rewards:
                reward += oppo_total_rewards
            elif self_total_rewards < oppo_total_rewards:
                reward += -self_total_rewards

        return next_state, reward, terminated, truncated, info


In [5]:
def get_reward_episode(model_name, env, T, n_episodes,is_print=False,is_reward=True):
    """
    is_reward: 是否需要重结算对手的收益
    用训练出的策略模型,在确定和随机的条件下,与分别环境进行n_episodes交互,分别计算各步累计收益的平均值,并绘制平均收益和平均的alpha随时间步的变化
    """
    # 初始化确定策略和随机策略的各步的平均累计收益(n_episdodes次交互的总和)
    self_rewards_sum = np.zeros(T)
    # 初始化确定策略和随机策略的各步的平均alpha
    self_alphas_sum = np.zeros(T)
    # 初始化对手的各步的平均累计收益(n_episdodes次交互的总和)
    oppo_rewards_sum = np.zeros(T)
    # 初始化对手的各步的alpha的总和
    oppo_alphas_sum = np.zeros(T)

    for episode_index in range(n_episodes):
        # 初始化确定策略每步自身和对手的累计收益
        self_reward_episode = np.zeros(T)
        oppo_reward_epsisode = np.zeros(T)
        # 重置环境，获取初始状态
        state, _ = env.reset()
        # 确定策略与环境进行交互
        for t in range(T):
            # 将状态转换为 PyTorch 张量，并添加批量维度
            state_tensor = th.tensor(state, dtype=th.float32).unsqueeze(0)
            # 交互
            action, _ = model_name.predict(state_tensor, deterministic=True)
            next_state, reward, terminated, truncated, info = env.step(action)
            # 更新确定策略的累计对手收益
            oppo_reward_epsisode[t] = sum(state[3*env.k:4*env.k]*state[4*env.k:5*env.k])
            # 更新确定策略的累计自身收益
            if t == 0:
                self_reward_episode[t] = reward
            else:
                self_reward_episode[t] = reward+self_reward_episode[t-1]
            
            # 更新确定策略的累计alpha
            self_alphas_sum[t] += action
            oppo_alphas_sum[t] += env.oppo_alpha

            # 与环境交互
            state = next_state
        # 加上确定策略的本次交互的累计收益
        self_rewards_sum += self_reward_episode
        oppo_rewards_sum += oppo_reward_epsisode
        if is_reward:
            # 根据最终状态中对手和自己的累计受益再次结算收益
            self_arms_n = state[:env.k]
            self_arms_rewards_mean = state[env.k:2*env.k]
            oppo_arm_n = state[3*env.k:4*env.k]
            oppo_arm_rewards_mean = state[4*env.k:5*env.k]
            self_total_rewards = sum(self_arms_n[i]*self_arms_rewards_mean[i] for i in range(env.k))
            oppo_total_rewards = sum(oppo_arm_n[i]*oppo_arm_rewards_mean[i] for i in range(env.k))
            if self_total_rewards > oppo_total_rewards:
                oppo_rewards_sum[T-1] += -oppo_total_rewards
            elif self_total_rewards < oppo_total_rewards:
                oppo_rewards_sum[T-1] += self_total_rewards
        if is_print:
            print('Episode:', episode_index + 1)

        
    # 计算episode内两人每步的平均累计收益
    self_rewards_mean = self_rewards_sum / n_episodes
    oppo_rewards_mean = oppo_rewards_sum / n_episodes
    # 计算episode内两人每步的平均alpha
    self_alphas_mean = self_alphas_sum / n_episodes
    oppo_alphas_mean = oppo_alphas_sum / n_episodes
    return self_rewards_mean, self_alphas_mean, oppo_rewards_mean, oppo_alphas_mean

进行多轮训练

按一定轮次的频率保存训练所得的策略

后续读取训练结果，绘出平均最终收益随训练轮次的变化情况


重点关注参数：
参数名      | 描述 |
| ----------- | ----------- |
| probs      | 多臂老虎机设定       |
| T      | 每轮拉臂的次数       |
| total_timesteps      | 总训练轮数       |
| check_freq      | 训练结果保存频率       |

In [6]:
# 多臂老虎机的基本设定
np.random.seed(1)
probs = np.random.rand(5)
formatted_probs = [f"{prob:.4f}" for prob in probs]
print("伯努利多臂老虎机的概率为：", formatted_probs)
T = 50

伯努利多臂老虎机的概率为： ['0.4170', '0.7203', '0.0001', '0.3023', '0.1468']


In [ ]:
# 训练smart_policy_v3
oppo_model = SAC.load("smart_UCB_v0")
smart_env_v3 = MAB_Smart(probs,T,oppo_model)
class SaveOnBestTrainingRewardCallback(BaseCallback):
    def __init__(self, check_freq: int, save_path: str, verbose: int = 1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self) -> None:
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            model_filename = f'model_{self.n_calls}.zip'
            model_path = os.path.join(self.save_path, model_filename)
            self.model.save(model_path)
            if self.verbose > 0:
                print(f"Saving model checkpoint to {model_filename}")
                print(f"Model {model_filename} has been saved.")
        return True
    
# 定义保存路径和检查频率
save_path1 = '/Users/fengyilong/Git/MAB_SAC/UCB_smart_v3'
check_freq = 2000

# 创建回调函数实例
callback1 = SaveOnBestTrainingRewardCallback(check_freq=check_freq, save_path=save_path1)
# 创建模型
smart_model_v3 = SAC("MlpPolicy", smart_env_v3, gamma=1, verbose=2)

total_timesteps = 100000

# 训练模型并使用回调函数
smart_model_v3.learn(total_timesteps, callback=callback1)

smart_model_v3.save("smart_UCB_v3")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | 17.5     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 275      |
|    time_elapsed    | 0        |
|    total_timesteps | 200      |
| train/             |          |
|    actor_loss      | -2.47    |
|    critic_loss     | 1.66     |
|    ent_coef        | 0.971    |
|    ent_coef_loss   | -0.0492  |
|    learning_rate   | 0.0003   |
|    n_updates       | 99       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | 22.5     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 211      |
|    time_elapsed    | 1        |
|    total_timesteps | 400      |
| train/             |

In [ ]:
# 计算平均累计收益
self_rewards_v3 = []
oppo_rewards_v3 = []
timesteps = []
n_episodes = 10000

# 处理策略
for step in range(check_freq, total_timesteps + 1, check_freq):
    print("解压:", step)
    model_filename = f'model_{step}.zip'
    model_path1 = os.path.join(save_path1, model_filename)

    
    # 加载模型
    smart_model_v3 = SAC.load(model_path1)
    # 计算平均最后时刻的累计收益
    temp_self_reward,_,temp_oppo_reward,_= get_reward_episode(smart_model_v3, smart_env_v3, T, n_episodes)
    
    # 记录结果
    self_rewards_v3.append(temp_self_reward[T-1])
    oppo_rewards_v3.append(temp_oppo_reward[T-1])
    timesteps.append(step)

# 定义要保存的数据
data = {
    'self_rewards_v3': self_rewards_v3,
    'oppo_rewards_v3': oppo_rewards_v3
}
# 保存数据到本地文件
with open('/Users/fengyilong/Git/MAB_SAC/UCB_smart_v3/data_v3.pkl', 'wb') as f:
    pickle.dump(data, f)

print("数据已保存到 /Users/fengyilong/Git/MAB_SAC/UCB_smart_v3/data_v3.pkl")

# 绘制确定策略平均累计收益随训练次数的变化
plt.figure(figsize=(10, 6))
plt.plot(timesteps, self_rewards_v3, label='Self Rewards v3')
plt.plot(timesteps, oppo_rewards_v3, label='Opponent Rewards v3')
plt.xlabel('Training Timesteps')
plt.ylabel('Average Cumulative Reward v3')
plt.title('Average Cumulative Reward vs Training Timesteps v3')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# 训练的最终模型
n_episodes = 50000
smart_model_name_v3 = SAC.load("smart_UCB_v3")

# 调用 get_reward_episode 函数,计算确定策略和随机策略的每步的平均累计收益
self_rewards_mean_v3, self_alphas_mean_v3, oppo_rewards_mean_v3, oppo_alphas_mean_v3 = get_reward_episode(smart_model_name_v3, smart_env_v3, T, n_episodes,1)

In [ ]:
# 定义要保存的数据
data = {
    'self_rewards_mean_v3': self_rewards_mean_v3,
    'self_alphas_mean_v3': self_alphas_mean_v3,
    'oppo_alphas_mean_v3': oppo_alphas_mean_v3,
    'oppo_rewards_mean_v3': oppo_rewards_mean_v3
}


# 保存数据到本地文件
with open('/Users/fengyilong/Git/MAB_SAC/UCB_smart_v3/data_inter_v3.pkl', 'wb') as f:
    pickle.dump(data, f)

print("数据已保存到 /Users/fengyilong/Git/MAB_SAC/UCB_smart_v3/data_inter_v3.pkl")


In [ ]:
# 从本地文件加载数据
with open('/Users/fengyilong/Git/MAB_SAC/UCB_smart_v3/data_inter_v3.pkl', 'rb') as f:
    loaded_data = pickle.load(f)
# 访问加载的数据
self_rewards_mean_v3 = loaded_data['self_rewards_mean_v3']
self_alphas_mean_v3 = loaded_data['self_alphas_mean_v3']
oppo_rewards_mean_v3 = loaded_data['oppo_rewards_mean_v3']
oppo_alphas_mean_v3 = loaded_data['oppo_alphas_mean_v3']


print("数据已从 /Users/fengyilong/Git/MAB_SAC/data_inter.pkl 加载")


# 绘制两种确定策略的每步的平均累计收益
plt.figure(figsize=(10, 6))
plt.plot(self_rewards_mean_v3, label='Self Rewards v3')
plt.plot(oppo_rewards_mean_v3, label='Oppoent Rewards v3')
plt.xlabel('Time Steps')
plt.ylabel('Average Cumulative Reward v3')
plt.title('Average Cumulative Reward vs Time Steps v3')
plt.legend()
plt.grid()
plt.show()


# 绘制两种确定策略每步的平均alpha
plt.figure(figsize=(10, 6))
plt.plot(self_alphas_mean_v3, label='Self Alpha v3')
plt.plot(oppo_alphas_mean_v3, label='Opponent Alpha v3')
plt.xlabel('Time Steps')
plt.ylabel('Average Alpha')
plt.title('Average Alpha vs Time Steps v3')
plt.legend()
plt.grid()
plt.show()